In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.prune as prune

from scipy import io
import os

/Users/goldenyoo/miniforge3/envs/mac_cpu/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mat_file = io.loadmat('/Users/goldenyoo/Library/Mobile Documents/com~apple~CloudDocs/BioCAS_prepare/Python_code/Data_center/one_dx/Calib_data_1.mat')


In [3]:
K1 = mat_file['K1']
A1 = mat_file['A1']

K2 = mat_file['K2']
A2 = mat_file['A2']

# K1 = np.swapaxes(K1,0,2)
# K1 = np.swapaxes(K1,1,2)
# A1 = np.swapaxes(A1,0,2)
# A1 = np.swapaxes(A1,1,2)

# K2 = np.swapaxes(K2,0,2)
# K2 = np.swapaxes(K2,1,2)
# A2 = np.swapaxes(A2,0,2)
# A2 = np.swapaxes(A2,1,2)


Y1 = mat_file['Y1']
Y2 = mat_file['Y2']

In [4]:
k1 = torch.FloatTensor(K1)

In [11]:
n_class = 22
n_hidden = 16
dtype = torch.float

class TextLSTM(nn.Module):
  def __init__(self):
    super(TextLSTM, self).__init__()

    self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden, dropout=0.3)
    self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
    self.b = nn.Parameter(torch.randn([n_class]).type(dtype))
    # self.Softmax = nn.Softmax(dim=1) # softmax 포함되어있다고 하지 않았나? CrossEntropyLoss에...

  def forward(self, hidden_and_cell, X):
    X = X.transpose(0, 1)
    # outputs, hidden = self.lstm(X, hidden_and_cell)
    # outputs = outputs[-1]  # 최종 예측 Hidden Layer
    print(X.size())
    outputs, (h_n,c_n) = self.lstm(X, hidden_and_cell)
    # print(h_n.size())
    outputs = h_n[-1]  # 최종 예측 Hidden Layer

    model = torch.mm(outputs, self.W) + self.b  # 최종 예측 최종 출력 층
    return model

In [15]:
model = TextLSTM()

module_1 = model.lstm

/Users/goldenyoo/miniforge3/envs/mac_cpu/lib/python3.8/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [18]:
list(module_1.named_parameters())[0][0]

'weight_ih_l0'